In [19]:
%load_ext autoreload
%autoreload 2

import json
import pysolr
import os
from dotenv import load_dotenv

load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [21]:
SOLR_DOCS_URL = "http://localhost:8983/solr/docuDOCS"  
JSON_FILE = "woo_docs.json" 

solr_rag_url = os.getenv("SOLR_RAG_URL")
username = os.getenv("SOLR_USER")
password = os.getenv("SOLR_PASSWORD")


In [22]:
solr_docs = pysolr.Solr(SOLR_DOCS_URL, always_commit=True, timeout=10)
solr_rag = pysolr.Solr(solr_rag_url, always_commit=True, timeout=10, auth=(username, password))

## Local files -> DocuDOCS collection

This piece of code uploads local files to SOLR. 

In [4]:
with open(JSON_FILE, "r", encoding="utf-8") as f:
    docs = json.load(f)

In [5]:
if isinstance(docs, dict):
    docs = [docs]

In [ ]:
to_add = []
for doc in docs:
    doc_id = str(doc["id"])  # ensure string id
    doc_date = doc.get("date")

    # Use 'date' as 'last_updated'
    doc["last_updated"] = doc_date
    # Check if doc already exists in Solr
    results = solr_docs.search(q=f"id:{doc_id}", fl="id,last_updated", rows=1)

    if not results:  
        # New doc → add
        to_add.append(doc)
        print(f"➕ Adding new doc {doc_id}")
    else:
        # Existing doc → compare last_updated
        existing = results.docs[0]
        existing_ts = existing.get("last_updated")
        if type(existing_ts) == list:
            existing_ts = existing_ts[0]
        
        if not existing_ts or doc_date > existing_ts:
            to_add.append(doc)
            print(f"♻️ Updating doc {doc_id}")
        else:
            print(f"✔️ Skipping unchanged doc {doc_id}")

# Send updates to Solr
if to_add:
    solr_docs.add(to_add)
    print(f"✅ Uploaded {len(to_add)} new/updated docs to rag_docs")
else:
    print("ℹ️ No new or updated docs to upload")

✔️ Skipping unchanged doc 1
✔️ Skipping unchanged doc 2
✔️ Skipping unchanged doc 3
✔️ Skipping unchanged doc 4
✔️ Skipping unchanged doc 5
✔️ Skipping unchanged doc 6
✔️ Skipping unchanged doc 7
✔️ Skipping unchanged doc 8
✔️ Skipping unchanged doc 9
✔️ Skipping unchanged doc 10
✅ Uploaded 10 new/updated docs to rag_docs


In [7]:
results = solr_docs.search("*:*", rows=50)

for r in results:
    print(r)

{'id': '1', 'title': 'WOO-verzoek inzake opvanglocaties AZC’s', 'municipality': 'Gemeente Wageningen', 'date': '2023-05-12T00:00:00Z', 'summary': ['Verzoek tot openbaarmaking van documenten over besluitvorming rondom uitbreiding van asielzoekerscentra en tijdelijke opvangplaatsen.'], 'text': 'Dit document bevat de volledige openbaarmaking van beleidsstukken, vergadernotulen en interne adviezen die betrekking hebben op de besluitvorming rondom de uitbreiding van asielzoekerscentra (AZC’s) binnen de gemeente Wageningen. Het doel van deze openbaarmaking is om transparantie te bieden over de afwegingen die hebben geleid tot de keuze voor specifieke locaties en de communicatie met bewoners, maatschappelijke organisaties en het COA.\n\nIn de voorbereidende fase is uitvoerig onderzocht welke locaties geschikt zouden kunnen zijn voor de tijdelijke en permanente opvang van asielzoekers. Hierbij zijn criteria gehanteerd zoals bereikbaarheid met openbaar vervoer, beschikbaarheid van voorzieningen

## docs -> docuRAG

In [23]:
from embeddings import build_items, encode_cpu, search_solr
from RAG import augmented_generation

### Docs -> embeddings -> SOLR

create embeddings for test documents and save them in SOLR

In [24]:
with open(JSON_FILE, "r", encoding="utf-8") as f:
    docs = json.load(f)

items = build_items(docs, chunk_size=300, overlap=30)
texts = [it["text"] for it in items]
emb = encode_cpu(texts)
for i, item in enumerate(items):
    item["emb"] = emb[i].tolist()
solr_rag.add(items)
print(f"Indexed {len(items)} chunks.")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]
INFO:pysolr:Finished 'https://solr.commonground.nu/solr/chunks/update/?commit=true' (post) with body '[{"vector_' in 1.026 seconds, with status 200


Indexed 16 chunks.


### User question -> SOLR top embeddings

In [25]:
# Check SOLR connection, print number of documents in the index
print("numFound:", solr_rag.search(q="*:*", rows=0).hits)

INFO:pysolr:Finished 'https://solr.commonground.nu/solr/chunks/select/?q=%2A%3A%2A&rows=0&wt=json' (get) with body '' in 0.278 seconds, with status 200


numFound: 16


In [26]:
all_data = solr_rag.search(q="*:*")
print(all_data)
for d in all_data:
    print(d)

INFO:pysolr:Finished 'https://solr.commonground.nu/solr/chunks/select/?q=%2A%3A%2A&wt=json' (get) with body '' in 0.163 seconds, with status 200


{'vector_id': ['1:2f904faa:0'], 'doc_id': '1', 'version': '2f904faa', 'municipality': ['Gemeente Wageningen'], 'date': ['2023-05-12T00:00:00Z'], 'title': ['WOO-verzoek inzake opvanglocaties AZC’s'], 'chunk_id': '1', 'text': ['WOO-verzoek inzake opvanglocaties AZC’s Verzoek tot openbaarmaking van documenten over besluitvorming rondom uitbreiding van asielzoekerscentra en tijdelijke opvangplaatsen. Dit document bevat de volledige openbaarmaking van beleidsstukken, vergadernotulen en interne adviezen die betrekking hebben op de besluitvorming rondom de uitbreiding van asielzoekerscentra (AZC’s) binnen de gemeente Wageningen. Het doel van deze openbaarmaking is om transparantie te bieden over de afwegingen die hebben geleid tot de keuze voor specifieke locaties en de communicatie met bewoners, maatschappelijke organisaties en het COA. In de voorbereidende fase is uitvoerig onderzocht welke locaties geschikt zouden kunnen zijn voor de tijdelijke en permanente opvang van asielzoekers. Hierbi

In [27]:
query = "Wat zegt het over verkeersveiligheid op de Vestdijk?"
results = search_solr(query, solr_rag)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: intfloat/multilingual-e5-small
Batches: 100%|██████████| 1/1 [00:00<00:00, 36.67it/s]
INFO:pysolr:Finished 'https://solr.commonground.nu/solr/chunks/select/' (post) with body 'q=%7B%21kn' in 0.064 seconds, with status 200


In [28]:
for r in results:
    print(r["score"], r["text"][:140])

0.9315195 ['en Vervoer, waarin wordt gewezen op de noodzaak om maatregelen gefaseerd door te voeren. Ook zijn er rapportages van externe adviesbureaus opgenomen, die met verkeersmodellen inzicht geven in de effecten op doorstroming en luchtkwaliteit. Uit deze rapportages blijkt dat de maatregelen naar verwachting leiden tot een afname van 25% van de autobewegingen en een meetbare verbetering van de luchtkwaliteit. De openbaarmaking omvat ook correspondentie met de provincie Noord-Brabant en Rijkswaterstaat, aangezien de herinrichting invloed heeft op verkeersstromen in de bredere regio. De documenten laten zien dat er intensief is afgestemd om knelpunten te voorkomen en om de plannen in lijn te brengen met provinciale mobiliteitsdoelen. Samenvattend bieden de openbaar gemaakte documenten een gedetailleerd beeld van de complexe afwegingen rondom verkeersveiligheid, leefbaarheid en economische belangen. Het plan voor de Vestdijk geldt inmiddels als voorbeeldproject binnen Eindhoven en vo

In [29]:
for r in results:
    print(r)

{'doc_id': '2', 'municipality': ['Gemeente Eindhoven'], 'date': ['2022-11-03T00:00:00Z'], 'title': ['WOO-besluit over verkeersveiligheidsplan'], 'chunk_id': '2', 'text': ['en Vervoer, waarin wordt gewezen op de noodzaak om maatregelen gefaseerd door te voeren. Ook zijn er rapportages van externe adviesbureaus opgenomen, die met verkeersmodellen inzicht geven in de effecten op doorstroming en luchtkwaliteit. Uit deze rapportages blijkt dat de maatregelen naar verwachting leiden tot een afname van 25% van de autobewegingen en een meetbare verbetering van de luchtkwaliteit. De openbaarmaking omvat ook correspondentie met de provincie Noord-Brabant en Rijkswaterstaat, aangezien de herinrichting invloed heeft op verkeersstromen in de bredere regio. De documenten laten zien dat er intensief is afgestemd om knelpunten te voorkomen en om de plannen in lijn te brengen met provinciale mobiliteitsdoelen. Samenvattend bieden de openbaar gemaakte documenten een gedetailleerd beeld van de complexe a

In [30]:
augmented_generation(results, query)

loading llm message
llm message: Doc 1: Doc id: 3a6d733f-9728-4d16-9af5-1375b1d9b106; Title: ['WOO-besluit over verkeersveiligheidsplan']
Date: ['2022-11-03T00:00:00Z']
Document: ['en Vervoer, waarin wordt gewezen op de noodzaak om maatregelen gefaseerd door te voeren. Ook zijn er rapportages van externe adviesbureaus opgenomen, die met verkeersmodellen inzicht geven in de effecten op doorstroming en luchtkwaliteit. Uit deze rapportages blijkt dat de maatregelen naar verwachting leiden tot een afname van 25% van de autobewegingen en een meetbare verbetering van de luchtkwaliteit. De openbaarmaking omvat ook correspondentie met de provincie Noord-Brabant en Rijkswaterstaat, aangezien de herinrichting invloed heeft op verkeersstromen in de bredere regio. De documenten laten zien dat er intensief is afgestemd om knelpunten te voorkomen en om de plannen in lijn te brengen met provinciale mobiliteitsdoelen. Samenvattend bieden de openbaar gemaakte documenten een gedetailleerd beeld van de c

{'answer': 'Het WOO-besluit over verkeersveiligheidsplan op de Vestdijk heeft als doel de doorstroming te verbeteren, de luchtkwaliteit te verhogen en de veiligheid van kwetsbare verkeersdeelnemers, zoals fietsers en voetgangers, te waarborgen. Uit metingen blijkt dat de concentratie fijnstof en stikstofdioxide op meerdere meetpunten boven de Europese normen uitkwam en dat de politie een bovengemiddeld aantal verkeersongevallen in dit gebied registreerde, waarvan een aanzienlijk deel met fietsers. De gemeenteraad koos voor een combinatie van maatregelen, waaronder een knip voor doorgaand verkeer, de aanleg van extra groenvoorzieningen en de verbreding van trottoirs en fietsstroken.',
 'sources': [{'id': 1,
   'title': 'WOO-besluit over verkeersveiligheidsplan',
   'date': '2022-11-03',
   'reason': 'verkeersveiligheid Vestdijk'},
  {'id': 2,
   'title': 'WOO-besluit over verkeersveiligheidsplan',
   'date': '2022-11-03',
   'reason': 'verkeersveiligheid Vestdijk'}]}